# Walkthrough downloading planet images

The following notebook incorporates tutorials on the Planet CLI from the following sources:
-  https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_python_client_introduction.ipynb
- https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/orders_api_tutorials/Planet_SDK_Orders_demo.ipynb

Please obtain your Planet API key from your Planet account before continuing.

In [1]:
from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import json
import requests
import time
import os
import shutil
import cv2 as cv
import rasterio
from rasterio.windows import Window
from pyproj import Transformer
import numpy as np
import os
from os import listdir
from os.path import isfile, join
np.random.seed(42)

import copy
import matplotlib
import matplotlib.pyplot as plt


import pyproj
import pickle as pkl
import geojson
import cv2
from PIL import Image, ImageEnhance# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

# *
The following code block requires ONE place to enter your API, on the bottom line paste your API where it says `PASTE API KEY HERE`.

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "HIDDEN")

# *
The following code block requires the file path to your geojson file, enter the file path and name of your region of interest.

In [3]:
with open("../Downloads/map.geojson") as f:
    geom = json.loads(f.read())

# *
The following code block is customizable.
- `geom_filter`: your geojson region of interset defined above.
- `clear_percent_filter`: percentage of clearness in each images
- `date_range_filter`: time period of study (`gt`: greater than, `lt`: less than)
- `cloud_cover_filter`: filter out clouds

In [4]:
item_types = ["PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=5, day=15, year=2023), lt = datetime(month=8, day=15, year=2023))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.5)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, cloud_cover_filter, date_range_filter])

In [5]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

# Search the Data API
# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our limit to the max of 500

In [6]:
async with Session() as sess:
    cl = DataClient(sess)
    items =  cl.run_search(search_id=request['id'], limit=500) 
    item_list = [i async for i in items]

Following block prints all images to be downloaded. (all images that match your filters and contain any match of coordinates in your provided geojson.

In [7]:
items = []
for item in item_list:
#     print(item['id'], item['properties']['item_type'])
    items.append(item['id'])
print(len(items))

500


In [17]:
item_list[0]['_permissions']

['assets.basic_analytic_4b:download',
 'assets.basic_analytic_4b_rpc:download',
 'assets.basic_analytic_4b_xml:download',
 'assets.basic_analytic_8b:download',
 'assets.basic_analytic_8b_xml:download',
 'assets.basic_udm2:download',
 'assets.ortho_analytic_4b:download',
 'assets.ortho_analytic_4b_sr:download',
 'assets.ortho_analytic_4b_xml:download',
 'assets.ortho_analytic_8b:download',
 'assets.ortho_analytic_8b_sr:download',
 'assets.ortho_analytic_8b_xml:download',
 'assets.ortho_udm2:download',
 'assets.ortho_visual:download']

In [8]:
#creates json for order
with open('results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

Following code requires your API key again .

In [10]:
#order details
from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')

auth = Auth.from_key(api_key)

Enter your API key:········


In [18]:
from planet import order_request

item_ids = items

products = [
    order_request.product(item_ids, 'basic_analytic_4b', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic')
]

request = order_request.build_request(
    'test_order_sdk_method_2', products=products, tools=tools)

SpecificationException: bundle - 'basic_analytic_4b' is not one of 'analytic_udm2', 'analytic_3b_udm2', 'analytic_8b_udm2', 'visual', 'basic_analytic_udm2', 'basic_analytic_8b_udm2', 'analytic_sr_udm2', 'analytic_8b_sr_udm2'.

# create order
Log in to planet at planet.com and check status of download.

**Note: this block appears to run indefinitely. By checking the status on your browser at planet.com, you will see when it has completed, and you can then stop this block and continue on. You can also check the directory in which this jupyter notebook is in and you will see a new folder with an obscure name consisting of numbers and characters.

In [12]:
from planet import reporting, Session, OrdersClient

# remember: "async def" to create the async coroutine
async def create_poll_and_download():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # create order via Orders client
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            await cl.wait(order['id'], callback=bar.update_state)

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'])

# remember: "await" to run the thing
await create_poll_and_download()

16:43 - order 3a4708ad-fac1-4b5a-8714-2583a2e160ce - state: running


ClientError: Maximum number of attempts (200) reached.